In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
PATH = 'drive/My Drive/DataWhale-零基础入门NLP - 新闻文本分类'
%cd drive/My Drive/DataWhale-零基础入门NLP - 新闻文本分类

/content/drive/My Drive/DataWhale-零基础入门NLP - 新闻文本分类


In [2]:
import torch
from torchtext import data
from torchtext import datasets
import random 
import numpy as np
import transformers
import logging
import random
import time

SEED = 2020
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

In [3]:
fold_num = 10
DATA_PATH = 'train_set.csv'
import pandas as pd

def all_data2fold(fold_num,num=10000):
    fold_data = []
    f = pd.read_csv(DATA_PATH,sep='\t',encoding='UTF-8',nrows=num)
    texts = f['text'].tolist()
    labels = f['label'].tolist()
    
    total = len(labels)
    index = list(range(total))
    np.random.shuffle(index)
    all_texts = []
    all_labels = []
    for i in index:
        all_texts.append(texts[i])
        all_labels.append(labels[i])
    label2id = {}
    for i in range(total):
        label = str(all_labels[i])
        if label not in label2id:
            label2id[label] = [i]
        else:
            label2id[label].append(i)
    all_index = [[] for _ in range(fold_num)]
    for label,data in label2id.items():
        batch_size = int(len(data)/fold_num)
        other = len(data) - batch_size*fold_num
        pre_batch_size = batch_size
        for i in range(fold_num):
            if i < other:
                cur_batch_size = batch_size +1;
            else: cur_batch_size = batch_size
            batch_data = [data[i*pre_batch_size+b] for b in range(cur_batch_size)]
            all_index[i].extend(batch_data)
            pre_batch_size = cur_batch_size
    
    batch_size = int(total/fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        num = len(all_index[fold])
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]
        
        if num > batch_size:
            fold_texts = texts[:batch_size]
            other_texts.extend(texts[batch_size:])
            fold_labels = labels[:batch_size]
            other_labels.extend(labels[batch_size:])
            other_num +=num-batch_size
        # 实现对orther_texts的利用
        elif num<batch_size:
            end = start+batch_size-num
            fold_texts = texts + other_texts[start:end]
            fold_labels = labels + other_labels[start:end]
            start = end
        else:
            fold_texts = texts
            fold_labels = labels
        assert batch_size == len(fold_labels)
        
        index = list(range(batch_size))
        np.random.shuffle(index)
        
        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])
        data = {'label':shuffle_fold_labels,'text':shuffle_fold_texts}
        fold_data.append(data)
    logging.info("Fold lens %s", str([len(data['label']) for data in fold_data]))
    return fold_data

fold_data = all_data2fold(10,100000)

2020-08-03 14:28:02,174 INFO: Fold lens [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]


In [4]:
fold_id = 9
train_texts = []
for i in range(0,fold_id):
    data = fold_data[i]
    train_texts.extend(data['text'])
logging.info('Total %d docs.' % len(train_texts))

2020-08-03 14:29:25,127 INFO: Total 90000 docs.


In [5]:
from gensim.models import Word2Vec

num_features = 100
num_workers = 8

train_texts = list(map(lambda x: list(x.split()),train_texts))
start = time.time()
model = Word2Vec(train_texts,workers=num_workers,size=num_features)
model.init_sims(replace=True)
# save model
model.save("word2vec.bin")
end = time.time()
logging.info("Running time %lfs",(end-start)/60.0)

2020-08-03 14:29:28,652 INFO: 'pattern' package not found; tag filters are not available for English
2020-08-03 14:29:35,704 INFO: collecting all words and their counts
2020-08-03 14:29:35,706 INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-03 14:29:36,958 INFO: PROGRESS: at sentence #10000, processed 9137963 words, keeping 5279 word types
2020-08-03 14:29:38,197 INFO: PROGRESS: at sentence #20000, processed 18316501 words, keeping 5670 word types
2020-08-03 14:29:39,447 INFO: PROGRESS: at sentence #30000, processed 27496347 words, keeping 5846 word types
2020-08-03 14:29:40,654 INFO: PROGRESS: at sentence #40000, processed 36497241 words, keeping 6004 word types
2020-08-03 14:29:41,844 INFO: PROGRESS: at sentence #50000, processed 45479632 words, keeping 6115 word types
2020-08-03 14:29:43,065 INFO: PROGRESS: at sentence #60000, processed 54650433 words, keeping 6203 word types
2020-08-03 14:29:44,276 INFO: PROGRESS: at sentence #70000, processed 636631

In [6]:
model.wv.save_word2vec_format('word2vec.txt', binary=False)

2020-08-03 14:54:29,888 INFO: storing 5644x100 projection weights into word2vec.txt
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model.wv.save_word2vec_format